In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time
from datetime import datetime
import random

In [2]:
# Loading csv files and combining them into dataframe
data_files=['../data/201901-citibike-tripdata.csv',
            '../data/201902-citibike-tripdata.csv',
            '../data/201903-citibike-tripdata.csv',
            '../data/201904-citibike-tripdata.csv',
            '../data/201905-citibike-tripdata.csv',
            '../data/201906-citibike-tripdata.csv',
            '../data/201907-citibike-tripdata.csv',
            '../data/201908-citibike-tripdata.csv',
            '../data/201909-citibike-tripdata.csv',
            '../data/201910-citibike-tripdata.csv',
            '../data/201911-citibike-tripdata.csv',
            '../data/201912-citibike-tripdata.csv']

dfs = [pd.read_csv(f).sample(frac=0.05, random_state = 42) for f in data_files]

df = pd.concat(dfs,ignore_index=True)

print(df.shape)

(1027583, 15)


In [3]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,574,2019-01-30 17:30:59.7870,2019-01-30 17:40:34.2530,359.0,E 47 St & Park Ave,40.755103,-73.974987,528.0,2 Ave & E 31 St,40.742909,-73.977061,33216,Subscriber,1993,1
1,709,2019-01-23 19:28:12.1530,2019-01-23 19:40:01.7030,493.0,W 45 St & 6 Ave,40.756800,-73.982912,3159.0,W 67 St & Broadway,40.774925,-73.982666,16092,Subscriber,1990,1
2,1993,2019-01-26 18:30:29.2160,2019-01-26 19:03:43.0780,503.0,E 20 St & Park Ave,40.738274,-73.987520,3709.0,W 15 St & 6 Ave,40.738046,-73.996430,34868,Subscriber,1977,1
3,392,2019-01-05 18:31:40.6190,2019-01-05 18:38:13.1980,161.0,LaGuardia Pl & W 3 St,40.729170,-73.998102,438.0,St Marks Pl & 1 Ave,40.727791,-73.985649,31405,Subscriber,1996,1
4,543,2019-01-23 15:24:24.9240,2019-01-23 15:33:28.8630,3496.0,1 Ave & E 110 St,40.792327,-73.938300,3671.0,E 81 St & 2 Ave,40.774779,-73.954275,35695,Subscriber,1998,1


In [4]:
print(df.shape)

(1027583, 15)


In [5]:
df.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id           float64
start station name          object
start station latitude     float64
start station longitude    float64
end station id             float64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                   int64
gender                       int64
dtype: object

In [6]:
df.isnull().sum()

tripduration                0
starttime                   0
stoptime                    0
start station id           16
start station name         16
start station latitude      0
start station longitude     0
end station id             16
end station name           16
end station latitude        0
end station longitude       0
bikeid                      0
usertype                    0
birth year                  0
gender                      0
dtype: int64

# Convert to datetime

In [7]:
df['starttime'] = pd.to_datetime(df['starttime'])
df['stoptime'] = pd.to_datetime(df['stoptime'])
df.rename(columns={'start station id':'start_station_id', 'start station name':'start_station_name',
                     'start station latitude':'start_station_latitude', 'start station longitude':'start_station_longitude',
                     'end station id':'end_station_id', 'end station name':'end_station_name',
                     'end station latitude':'end_station_latitude', 'end station longitude':'end_station_longitude'},
             inplace=True)
df.dropna(inplace=True)
df['start_station_id'] = df['start_station_id'].astype(int)
df['end_station_id'] = df['end_station_id'].astype(int)
df['date'] = df['starttime'].dt.strftime('%Y-%m-%d')
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.strftime('%m')
df['day'] = df['date'].dt.strftime('%d')
df['hour'] = df['starttime'].dt.hour
df['dayofweek'] = df['date'].dt.weekday+1   # Monday:1 ~ Sunday: 7
df['Isweekday'] = [0 if 6<=x<=7 else 1 for x in df['dayofweek']]
df['age'] = 2020 - df['birth year']

In [8]:
df.head()

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,...,usertype,birth year,gender,date,month,day,hour,dayofweek,Isweekday,age
0,574,2019-01-30 17:30:59.787,2019-01-30 17:40:34.253,359,E 47 St & Park Ave,40.755103,-73.974987,528,2 Ave & E 31 St,40.742909,...,Subscriber,1993,1,2019-01-30,01,30,17,3,1,27
1,709,2019-01-23 19:28:12.153,2019-01-23 19:40:01.703,493,W 45 St & 6 Ave,40.756800,-73.982912,3159,W 67 St & Broadway,40.774925,...,Subscriber,1990,1,2019-01-23,01,23,19,3,1,30
2,1993,2019-01-26 18:30:29.216,2019-01-26 19:03:43.078,503,E 20 St & Park Ave,40.738274,-73.987520,3709,W 15 St & 6 Ave,40.738046,...,Subscriber,1977,1,2019-01-26,01,26,18,6,0,43
3,392,2019-01-05 18:31:40.619,2019-01-05 18:38:13.198,161,LaGuardia Pl & W 3 St,40.729170,-73.998102,438,St Marks Pl & 1 Ave,40.727791,...,Subscriber,1996,1,2019-01-05,01,05,18,6,0,24
4,543,2019-01-23 15:24:24.924,2019-01-23 15:33:28.863,3496,1 Ave & E 110 St,40.792327,-73.938300,3671,E 81 St & 2 Ave,40.774779,...,Subscriber,1998,1,2019-01-23,01,23,15,3,1,22


In [9]:
df.to_csv('final.csv')